In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st
import matplotlib.pyplot as plt
import matlab.engine
eng = matlab.engine.start_matlab()

#greedy copula segmentation
def GCS(x,copula_family,lbd):
    K = 1
    seg = []
    seg.append(x)
    while K > 0:
        print(K)
        LL_all = np.zeros((K,len(x)))
        for j in range(K):
            n = len(seg[j])
            LL_segorig = LL(seg[j],copula_family,lbd)
            for k in range(2,n-3):
                print(k,end='\r')
                LL_all[j,k] = LL(seg[j][:k,:],copula_family,lbd) + LL(seg[j][k:,:],copula_family,lbd) - LL_segorig
            if K == 1:
                LL_all_1 = LL_all
            elif K == 2:
                LL_all_2 = LL_all

        if np.max(LL_all) > 0:
            j_star, k_star = np.where(LL_all == np.max(LL_all))
            if j_star.item() == (K-1):
                seg.append(seg[K-1][k_star.item()+1:,:])
                seg[K-1] = seg[K-1][:k_star.item()+1,:]
                K = K + 1
            else:
                K = -1
        else:
            K = -1
    return seg, LL_all_1, LL_all_2

def LL(x,copula_family,lbd):
    #marginal distribution fitting
    pd1 = st.gamma.fit(x[:,0])
    dist1 = st.gamma(a=pd1[0],loc=pd1[1],scale=pd1[2])
    cdf1 = dist1.cdf(x[:,0])
    var1 = np.var(x[:,0])

    pd2 = st.lognorm.fit(x[:,1])
    dist2 = st.lognorm(s=pd2[0],loc=pd2[1],scale=pd2[2])
    cdf2 = dist2.cdf(x[:,1])
    var2 = np.var(x[:,1])

    #copula fitting
    cdfs = eng.transpose(matlab.double([cdf1,cdf2]))
    copula_paramhat = eng.copulafit(copula_family, cdfs)

    #loglikelihood
    loglikelihood = eng.sum(eng.log(eng.copulapdf(copula_family,cdfs,copula_paramhat))) - lbd / (var1 + var2)

    return loglikelihood

In [2]:
df = pd.read_table('../../data/Benchmark/toyproblem.csv', sep = ',')
x = np.array(df)
copula_family = 'Clayton'
lbd = 100

In [4]:
K = 1
seg = []
seg.append(x)
LL_all = np.zeros((K,len(x)))
j = 0
n = len(seg[j])
LL_segorig = LL(seg[j],copula_family,lbd)

In [12]:
%%time
for k in range(2,n-3):
    LL_all[j,k] = LL(seg[j][:k,:],copula_family,lbd) + LL(seg[j][k:,:],copula_family,lbd) - LL_segorig

CPU times: total: 3min 51s
Wall time: 4min 8s


In [18]:
%%time
result_pre = [test(seg[j],k,True,copula_family,lbd) for k in range(2,n-3)]
result_post = [test(seg[j],k,False,copula_family,lbd) for k in range(2,n-3)]
results = np.array(result_pre) - np.array(result_post) - LL_segorig

CPU times: total: 3min 53s
Wall time: 4min 10s


In [15]:
%%time
results = [test(seg[j],k,True,copula_family,lbd) + test(seg[j],k,False,copula_family,lbd) - LL_segorig for k in range(2,n-3)]

CPU times: total: 3min 53s
Wall time: 4min 9s


In [7]:
def test(seg,k,pre,copula_family,lbd):
    if pre == True:
        x = seg[:k]
    else:
        x = seg[k:]
        
    #marginal distribution fitting
    pd1 = st.gamma.fit(x[:,0])
    dist1 = st.gamma(a=pd1[0],loc=pd1[1],scale=pd1[2])
    cdf1 = dist1.cdf(x[:,0])
    var1 = np.var(x[:,0])

    pd2 = st.lognorm.fit(x[:,1])
    dist2 = st.lognorm(s=pd2[0],loc=pd2[1],scale=pd2[2])
    cdf2 = dist2.cdf(x[:,1])
    var2 = np.var(x[:,1])

    #copula fitting
    cdfs = eng.transpose(matlab.double([cdf1,cdf2]))
    copula_paramhat = eng.copulafit(copula_family, cdfs)

    #loglikelihood
    loglikelihood = eng.sum(eng.log(eng.copulapdf(copula_family,cdfs,copula_paramhat))) - lbd / (var1 + var2)

    return loglikelihood